In [61]:
from nltk.corpus import wordnet
def getSynset(wnid):
    pos = wnid[0]
    offset = wnid[1:]
    return wordnet._synset_from_pos_and_offset(pos,int(offset))
def getWordnetID(syn):
    return syn.pos()+('%08d' % syn.offset())

In [41]:
#获取openimage filtered synset列表
f = open("./SynId_init.txt")
syns = []
for synid in f:
    tmpid = synid.strip('\n')
    syn = getSynset(tmpid)
    syns.append(syn)

In [66]:
#获取imagenet attribute synset列表
f_imagenet_attr = open("SynsetswithAttribute.txt")
syns_imagenet_attr = []
for synid in f_imagenet_attr:
    synid = synid.strip('\n')
    syns_imagenet_attr.append(getSynset(synid))

In [67]:
#获取imagenet classfication synset列表
f_imagenet_label = open("synsets_ImageNet2012.txt")
syns_imagenet_label = []
for synid in f_imagenet_label:
    synid = synid.strip('\n')
    syns_imagenet_label.append(getSynset(synid))

In [73]:
#合并三个来源的synset
syns_union = set(syns) | set(syns_imagenet_attr) | set(syns_imagenet_label)
print 'openimaget synset num is ' + str(len(syns))
print 'imagenet attribute num is ' + str(len(syns_imagenet_attr))
print 'imagenet label num is ' + str(len(syns_imagenet_label))
print 'union synset num is ' + str(len(syns_union))

openimaget synset num is 872
imagenet attribute num is 384
imagenet label num is 1000
union synset num is 2032


In [77]:
#计算每个synset的子节点,记录只要一个子节点的节点的子节点编号
c = list(syns_union)
syns_union_child = {}
import numpy as np
childs = np.zeros(len(syns_union))
for i in range(len(syns_union)):
    for j in range(len(syns_union)):
        if i != j:
            if syns_union[j] in syns_union[i].hyponyms():
                childs[i] += 1  
                syns_union_child[i] = j

In [80]:
#找出有0,1，多个子节点的synset
print 'total num = ' + str(len(childs))
zero_num = 0
zero_list = []
one_num = 0
one_list = []
one_child = []
other_num = 0
for i,x in enumerate(childs):
    if x == 0:
        zero_num += 1
        zero_list.append(i)
    elif x == 1:
        one_num += 1
        one_list.append(i)
        one_child.append(syns_union_child[i])
    else:
        other_num += 1
print 'zero num = ' + str(zero_num)
print 'one num = ' + str(one_num)
print 'other num = ' + str(other_num)

total num = 2032
zero num = 1771
one num = 112
other num = 149


In [94]:
#输出不含子节点，加上含一个子节点，去掉一个子节点的子节点
print 'zero child synset num is %d' % len(zero_list)
print 'one child synset num is %d' % len(one_list)
print 'one child synset child num is %d' % len(one_child)
syns_union_filer = set(zero_list) | set(one_list)
print 'the union of zero and one child sysnet is %d' % len(syns_union_filer)
syns_union_filer = list(syns_union_filer - set(one_child))
print 'above num remove one child synset is %d' % len(syns_union_filer)
wf = open("course_union_synid.txt", "wb")
for i in syns_union_filer:
    synid = getWordnetID(syns_union[i])
    wf.write(synid + '\n')

zero child synset num is 1771
one child synset num is 112
one child synset child num is 112
the union of zero and one child sysnet is 1883
above num remove one child synset is 1778
